In [263]:
import pandas as pd
import os
import sys
import pickle
import numpy as np
import datetime

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve

import xgboost as xgb
import lightgbm as lgb

In [264]:
path = r'C:\Users\KILO\DataAnalysis\tianci'

In [265]:
traindata_offline = pd.read_csv(path + r'\ccf_offline_stage1_train.csv')
traindata_online = pd.read_csv(path + r'\ccf_online_stage1_train.csv')
test_data = pd.read_csv(path + r'\ccf_offline_stage1_test_revised.csv')

In [266]:
traindata_offline.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [267]:
traindata_online.head()

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,20160513.0,NaN
1,13740231,34805,1,NaN,NaN,NaN,20160321.0
2,14336199,18907,0,NaN,NaN,NaN,20160618.0
3,14336199,18907,0,NaN,NaN,NaN,20160618.0
4,14336199,18907,0,NaN,NaN,NaN,20160618.0


In [268]:
test_data.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,NaN,20160706
2,2166529,7113,6928,200:20,5.0,20160727
3,2166529,7113,1808,100:10,5.0,20160727
4,6172162,7605,6500,30:1,2.0,20160708


In [269]:
traindata_offline.describe()

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date
count,1.754884e+06,1.754884e+06,1.053282e+06,1.648881e+06,1.053282e+06,7.769840e+05
mean,3.689255e+06,4.038808e+03,6.815398e+03,2.361636e+00,2.016032e+07,2.016040e+07
std,2.123428e+06,2.435963e+03,4.174276e+03,3.483974e+00,1.771416e+02,1.621500e+02
min,4.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,2.016010e+07,2.016010e+07
25%,1.845052e+06,1.983000e+03,2.840000e+03,0.000000e+00,2.016013e+07,2.016031e+07
50%,3.694446e+06,3.532000e+03,7.430000e+03,0.000000e+00,2.016032e+07,2.016042e+07
75%,5.528759e+06,6.329000e+03,1.032300e+04,3.000000e+00,2.016051e+07,2.016052e+07
max,7.361032e+06,8.856000e+03,1.404500e+04,1.000000e+01,2.016062e+07,2.016063e+07


In [270]:
traindata_offline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
User_id          int64
Merchant_id      int64
Coupon_id        float64
Discount_rate    object
Distance         float64
Date_received    float64
Date             float64
dtypes: float64(4), int64(2), object(1)
memory usage: 93.7+ MB


# 空值数据

In [271]:
null_pct = traindata_offline.isnull().sum()/len(traindata_offline)
null_pct

User_id          0.000000
Merchant_id      0.000000
Coupon_id        0.399800
Discount_rate    0.399800
Distance         0.060405
Date_received    0.399800
Date             0.557245
dtype: float64

In [272]:
buy_without_coupon_pct = (null_pct.Date - null_pct.Coupon_id)/null_pct.Date
buy_without_coupon_pct

0.28254218222722166

Coupon_id，Discount_rate和Date_rec都有接近40%的空值，说明训练数据集中有40%的人没有领优惠券，而在购买了的商品的用户中,有28%的用户没有使用优惠券

# 重复数据

In [273]:
len(traindata_offline[traindata_offline.duplicated()])

37893

In [274]:
traindata_offline = traindata_offline.drop_duplicates()

In [275]:
traindata_offline.shape

(1716991, 7)

所有数据中总共有37893条重复数据，重复的数据对于结果预测是没有意义的，所以进行删除

# 数据转化

线下数据中，discount_rate列的数据是用满减的形式表述的，不利于算法进行计算，所以需要尽心拆分以及计算出折扣比率

转化折扣率函数

In [276]:
def convert_rate(row):
    if row == 'NaN':
        return 1.0
    elif ':' in str(row):
        rows = row.split(':')
        return round(1 - float(rows[1]) / float(rows[0]), 2)
    else:
        return float(row)

获得折扣类型

In [277]:
def get_discount_type(row):
    if row == 'null':
        return 'null'
    elif ':' in str(row):
        return 1
    else:
        return 0

获得满，减具体数值

In [278]:
def get_discount_man(row):
    if ':' in str(row):
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def get_discount_jian(row):
    if ':' in str(row):
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

In [279]:
def process_data(df):
    
# convert discount_rate
    df['discount_rate'] = df['Discount_rate'].apply(convert_rate)
    df['discount_man'] = df['Discount_rate'].apply(get_discount_man)
    df['discount_jian'] = df['Discount_rate'].apply(get_discount_jian)
    df['discount_type'] = df['Discount_rate'].apply(get_discount_type)
    print(df['discount_rate'].unique())
    
# convert distance
    df['distance'] = df['Distance'].fillna(-1).astype(int)
    print(df['distance'].unique())
    return df

In [280]:
traindata_offline = process_data(traindata_offline)
test_data = process_data(test_data)

[ nan 0.87 0.95 0.9  0.83 0.8  0.5  0.85 0.75 0.67 0.93 0.7  0.6  0.97
 0.98 0.99 0.33 0.2  0.4 ]
[ 0  1 -1  2 10  4  7  9  3  5  6  8]
[0.83 0.9  0.97 0.8  0.95 0.75 0.98 0.5  0.87 0.6  0.67 0.7  0.85 0.33
 0.94 0.93 0.99]
[ 1 -1  5  2  0 10  3  6  7  4  9  8]


In [281]:
traindata_offline.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_rate,discount_man,discount_jian,discount_type,distance
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,NaN,0,0,0,0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,0.87,150,20,1,1
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0.95,20,1,1,0
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0.95,20,1,1,0
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN,0.95,20,1,1,0


In [282]:
traindata_offline.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1716991 entries, 0 to 1754883
Data columns (total 12 columns):
User_id          int64
Merchant_id      int64
Coupon_id        float64
Discount_rate    object
Distance         float64
Date_received    float64
Date             float64
discount_rate    float64
discount_man     int64
discount_jian    int64
discount_type    int64
distance         int32
dtypes: float64(5), int32(1), int64(5), object(1)
memory usage: 163.7+ MB


把时间数据格式转换为时间格式

In [283]:
traindata_offline['date_received'] = pd.to_datetime(traindata_offline['Date_received'], format='%Y%m%d', errors='ignore')
traindata_offline['date_received'].head()

0          NaT
1   2016-05-28
2   2016-02-17
3   2016-03-19
4   2016-06-13
Name: date_received, dtype: datetime64[ns]

In [284]:
test_data['date_received'] = pd.to_datetime(test_data['Date_received'], format='%Y%m%d', errors='ignore')
test_data['date_received'].head()

0   2016-07-12
1   2016-07-06
2   2016-07-27
3   2016-07-27
4   2016-07-08
Name: date_received, dtype: datetime64[ns]

In [285]:
traindata_offline['Date'] = pd.to_datetime(traindata_offline['Date'], format='%Y%m%d', errors='ignore')
traindata_offline['Date'].head()

0   2016-02-17
1          NaT
2          NaT
3          NaT
4          NaT
Name: Date, dtype: datetime64[ns]

添加时间数据的星期特征

In [286]:
traindata_offline['week_day'] = traindata_offline['date_received'].dt.weekday + 1
traindata_offline['week_day'].head()

0    NaN
1    6.0
2    3.0
3    6.0
4    1.0
Name: week_day, dtype: float64

In [287]:
test_data['week_day'] = test_data['date_received'].dt.weekday + 1
test_data['week_day'].head()

0    2
1    3
2    3
3    3
4    5
Name: week_day, dtype: int64

In [288]:
traindata_offline['weekday_type'] = traindata_offline['week_day'].apply(lambda x : 1 if x in [6, 7] else 0)
traindata_offline['weekday_type'].head()                                                                       

0    0
1    1
2    0
3    1
4    0
Name: weekday_type, dtype: int64

In [289]:
test_data['weekday_type'] = test_data['week_day'].apply(lambda x : 1 if x in [6, 7] else 0)
test_data['weekday_type'].head()

0    0
1    0
2    0
3    0
4    0
Name: weekday_type, dtype: int64

对weekday数据进行one-hot编码

In [290]:
week_cols = ['week_' + str(i) for i in range(1, 8)]
tmpdf = pd.get_dummies(traindata_offline['week_day'].replace('nan', np.nan))
traindata_offline[week_cols] = tmpdf
traindata_offline[week_cols].head()

,week_1,week_2,week_3,week_4,week_5,week_6,week_7
0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0
2,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0


In [291]:
tmpdf = pd.get_dummies(test_data['week_day'].replace('nan', np.nan))
test_data[week_cols] = tmpdf
test_data[week_cols].head()

,week_1,week_2,week_3,week_4,week_5,week_6,week_7
0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0


In [292]:
def label(row):
    if pd.isnull(row['date_received']):
        return -1
    if pd.notnull(row['Date']):
        td = row['Date'] -  row['date_received']
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0
traindata_offline['label'] = traindata_offline.apply(label, axis = 1)

In [293]:
traindata_offline.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_rate,discount_man,discount_jian,...,week_day,weekday_type,week_1,week_2,week_3,week_4,week_5,week_6,week_7,label
0,1439408,2632,NaN,NaN,0.0,NaN,2016-02-17,NaN,0,0,...,NaN,0,0,0,0,0,0,0,0,-1
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaT,0.87,150,20,...,6.0,1,0,0,0,0,0,1,0,0
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaT,0.95,20,1,...,3.0,0,0,0,1,0,0,0,0,0
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaT,0.95,20,1,...,6.0,1,0,0,0,0,0,1,0,0
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaT,0.95,20,1,...,1.0,0,1,0,0,0,0,0,0,0


In [294]:
df = traindata_offline[traindata_offline['label'] != -1].copy()
train = df[(df['date_received'] < datetime.datetime.strptime('2016-05-16',"%Y-%m-%d"))].copy()
valid = df[(df['date_received'] >= datetime.datetime.strptime('2016-05-16',"%Y-%m-%d")) & (df['date_received'] <=datetime.datetime.strptime('2016-06-15',"%Y-%m-%d"))].copy()

In [295]:
train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_rate,discount_man,discount_jian,...,week_day,weekday_type,week_1,week_2,week_3,week_4,week_5,week_6,week_7,label
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaT,0.95,20,1,...,3.0,0,0,0,1,0,0,0,0,0
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaT,0.95,20,1,...,6.0,1,0,0,0,0,0,1,0,0
7,1832624,3381,7610.0,200:20,0.0,20160429.0,NaT,0.90,200,20,...,5.0,0,0,0,0,0,1,0,0,0
8,2029232,3381,11951.0,200:20,1.0,20160129.0,NaT,0.90,200,20,...,5.0,0,0,0,0,0,1,0,0,0
16,2223968,3381,9776.0,10:5,2.0,20160129.0,NaT,0.50,10,5,...,5.0,0,0,0,0,0,1,0,0,0


In [296]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 764065 entries, 2 to 1754880
Data columns (total 23 columns):
User_id          764065 non-null int64
Merchant_id      764065 non-null int64
Coupon_id        764065 non-null float64
Discount_rate    764065 non-null object
Distance         690665 non-null float64
Date_received    764065 non-null float64
Date             47227 non-null datetime64[ns]
discount_rate    764065 non-null float64
discount_man     764065 non-null int64
discount_jian    764065 non-null int64
discount_type    764065 non-null int64
distance         764065 non-null int32
date_received    764065 non-null datetime64[ns]
week_day         764065 non-null float64
weekday_type     764065 non-null int64
week_1           764065 non-null uint8
week_2           764065 non-null uint8
week_3           764065 non-null uint8
week_4           764065 non-null uint8
week_5           764065 non-null uint8
week_6           764065 non-null uint8
week_7           764065 non-null uint8
la

In [297]:
original_feature = ['discount_rate','discount_man', 'discount_jian','distance', 'week_day', 'weekday_type'] + week_cols
model = SGDClassifier(#lambda:
    loss='log',
    penalty='elasticnet',
    fit_intercept=True,
    max_iter=100,
    shuffle=True,
    alpha = 0.01,
    l1_ratio = 0.01,
    n_jobs=1,
    class_weight=None
)
model.fit(train[original_feature], train['label'])

SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.01, learning_rate='optimal', loss='log', max_iter=100,
              n_iter_no_change=5, n_jobs=1, penalty='elasticnet', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [298]:
print(model.score(valid[original_feature], valid['label']))

0.9096266174340691


In [299]:
with open('1_model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('1_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [300]:
# test prediction for submission
y_test_pred = model.predict_proba(test_data[original_feature])
dftest1 = test_data[['User_id','Coupon_id','Date_received']].copy()
dftest1['label'] = y_test_pred[:,1]
dftest1.to_csv('submit1.csv', index=False, header=False)
dftest1.head()

,User_id,Coupon_id,Date_received,label
0,4129537,9983,20160712,0.116078
1,6949378,3429,20160706,0.166846
2,2166529,6928,20160727,0.005493
3,2166529,1808,20160727,0.019487
4,6172162,6500,20160708,0.079031


In [301]:
dftest1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113640 entries, 0 to 113639
Data columns (total 4 columns):
User_id          113640 non-null int64
Coupon_id        113640 non-null int64
Date_received    113640 non-null int64
label            113640 non-null float64
dtypes: float64(1), int64(3)
memory usage: 3.5 MB


## 预测及结果评价

这边对每个coupon预测的结果计算AUC，再对所有的coupon做平均。计算AUC的时候，如果label只有一类，就直接跳过，因为AUC无法计算。

In [302]:
y_valid_pred = model.predict_proba(valid[original_feature])
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]
valid1.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_rate,discount_man,discount_jian,...,weekday_type,week_1,week_2,week_3,week_4,week_5,week_6,week_7,label,pred_prob
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaT,0.87,150,20,...,1,0,0,0,0,0,1,0,0,0.021795
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaT,0.95,20,1,...,0,1,0,0,0,0,0,0,0,0.147014


In [303]:
# avgAUC calculation
vg = valid1.groupby(['Coupon_id'])
aucs = []
for i in vg:
    tmpdf = i[1] 
    if len(tmpdf['label'].unique()) != 2:
        continue
    fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred_prob'], pos_label=1)
    aucs.append(auc(fpr, tpr))
print(np.average(aucs))

0.5339547183607439
